In [21]:
from google.colab import drive
drive.mount('/content/drive') #shift + Enter

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
from keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D, Input, Activation, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
root_path=r'/content/drive/MyDrive/dataset/fer'
emotions = os.listdir(root_path)

In [3]:
import cv2

In [4]:
happy_items=os.listdir(os.path.join(root_path, 'happy'))
len(happy_items)

1774

In [5]:
emotions

['sad', 'neutral', 'happy']

In [6]:
print(emotions.index('neutral'))

1


In [7]:
from keras.utils import to_categorical
import numpy as np

In [8]:
train_x = []
train_y = []

for emotion in emotions:
  full_path = os.path.join(root_path,emotion)
  imgs = os.listdir(full_path)
  for img in imgs:
    img_path = os.path.join(full_path, img)
    vector_img = image.load_img(img_path, target_size=(224,224))
    vector_img = image.img_to_array(vector_img)
    vector_img = preprocess_input(vector_img)
    train_x.append(vector_img)
    train_y.append(emotions.index(emotion))

train_x = np.asarray(train_x)
train_y = to_categorical(train_y)

In [9]:
train_x.shape, train_y.shape

((4254, 224, 224, 3), (4254, 3))

In [10]:
from keras.models import Model
from sklearn.model_selection import train_test_split

In [11]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=7)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=7)

In [12]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((2722, 224, 224, 3), (851, 224, 224, 3), (2722, 3), (851, 3))

In [13]:
base_model=ResNet50(input_shape=(224,224,3))
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
for layer in base_model.layers:
    layer.trainable=False
x=base_model.get_layer('avg_pool').output
x=Dense(3)(x)

model_save_path=r'/content/drive/MyDrive/saved_models'
model_checkpoint_callback=ModelCheckpoint(model_save_path, monitor='loss', save_best_only=True, mode='min')
model = Model(inputs=base_model.inputs, outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=50, batch_size=32, callbacks=[model_checkpoint_callback])

Epoch 1/50
86/86 [==============================] - 47s 365ms/step - loss: 4.8413 - accuracy: 0.4276 - val_loss: 4.9706 - val_accuracy: 0.4317
Epoch 2/50
86/86 [==============================] - 11s 129ms/step - loss: 4.8971 - accuracy: 0.4357 - val_loss: 4.9468 - val_accuracy: 0.4347
Epoch 3/50
86/86 [==============================] - 11s 124ms/step - loss: 5.0333 - accuracy: 0.4379 - val_loss: 5.0414 - val_accuracy: 0.4464
Epoch 4/50
86/86 [==============================] - 11s 132ms/step - loss: 5.1043 - accuracy: 0.4398 - val_loss: 5.1360 - val_accuracy: 0.4435
Epoch 5/50
86/86 [==============================] - 11s 128ms/step - loss: 5.1694 - accuracy: 0.4401 - val_loss: 5.1597 - val_accuracy: 0.4376
Epoch 6/50
53/86 [=================>............] - ETA: 3s - loss: 5.3030 - accuracy: 0.4269

In [ ]:
model.load_weights(model_save_path)
test_loss, test_acc = model.evaluate(test_x, test_y)

print('test acc:', test_acc)